TODO :
 - reorder cells to facilitate testing part
 - do the dataprocess only on the train dataset

In [10]:
%pip install pandas==1.5.1
%pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import torch
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets
from torchvision.transforms.functional import hflip
from torch.utils.data import Dataset,DataLoader
from torch.optim import Adam
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from matplotlib import image as mpimg
import re
from PIL import Image
import pandas as pd
import datetime
from torchmetrics import TweedieDevianceScore

import math
from torch import default_generator, randperm
from torch._utils import _accumulate
from torch.utils.data.dataset import Subset

from sklearn.metrics import fbeta_score, make_scorer, mean_poisson_deviance
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from IPython.display import display
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
import sklearn
from sklearn.metrics import mean_squared_error
from sklearn.metrics import fbeta_score, make_scorer



In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

PATH='/content/drive/MyDrive/MLproject/'

# Tabular data
with open(PATH+"listings.pkl", "rb") as fh:
    data = pickle.load(fh)
with open(PATH+"test_listings_overall.pkl", "rb") as fh:
    data_test = pickle.load(fh)

# Image data
images_data_train = pd.read_csv(PATH + "images_vectorized_2.csv")
images_data_test = pd.read_csv(PATH + "images_vectorized_test.csv")

cuda


In [16]:
# Text data
with open(PATH+"text_features.pkl", "rb") as fh:
    text_train = pickle.load(fh)

In [ ]:
images_data_train

,Property Reference Id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,Demand
0,0002.61.6915,-0.221832,6.024265,5.953249,-6.890490,3.063627,-0.948176,-0.362961,-1.883066,0.872796,6.887189,-9.850289,-0.950801,-2.359023,-7.314687,9.682235,-2.985931,2.0
1,0002.61.6915,0.609485,6.242800,5.783226,-6.939771,3.479691,-0.874210,0.335041,-1.675094,0.111615,6.779986,-9.572109,-0.092465,-2.760837,-7.171674,9.206720,-2.622147,2.0
2,0002.61.6915,1.398565,6.613178,6.145263,-7.383204,3.698415,-2.127483,0.909472,-2.784360,-0.298863,7.241723,-10.057664,0.637905,-3.071800,-7.520823,9.508305,-3.589651,2.0
3,0012.61.9682,0.732772,7.525365,6.753076,-8.926143,5.028864,-3.052044,0.783926,-4.661302,0.409809,7.880892,-12.079361,0.409625,-3.946983,-8.715199,11.561431,-4.128768,17.0
4,0012.61.9682,-1.362888,5.466207,5.663973,-6.751781,2.632468,-2.109637,-0.573149,-2.908666,1.750561,6.727478,-9.977320,-1.542024,-1.799264,-7.152509,10.005502,-2.396945,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5838,9995.50.8394,2.686135,10.535210,8.687286,-11.613550,6.980347,-1.652806,1.780055,0.754969,-1.321127,10.430278,-15.639960,2.439432,-6.165233,-11.678176,13.718158,-2.538267,5.0
5839,9995.50.8394,0.648810,8.603827,7.652774,-9.628755,4.941155,0.387756,0.320485,1.968812,-0.037644,9.185239,-13.673468,0.599522,-4.164209,-10.169299,12.084097,-1.126678,5.0
5840,9995.50.8394,1.953830,9.546097,7.472683,-9.778398,7.437314,-3.499745,4.520870,8.898790,-1.808473,8.530967,-13.685159,3.585144,-6.815229,-10.193208,12.663684,7.011076,5.0
5841,9995.50.8394,-1.864435,7.370160,6.550863,-8.974305,5.658174,-1.539836,-0.353800,3.408514,2.730169,7.720353,-13.286383,-1.674972,-4.154854,-8.953053,13.140576,2.727453,5.0


# Dataset

In [ ]:
data.columns

Index(['Are Pets Allowed', 'Category Idx', 'Day of Advertisement Created',
       'Day of Date Available From', 'Demand', 'Floor', 'Geo Canton',
       'Geo City', 'Geo Zip', 'Has Balcony', 'Has Cabletv', 'Has Elevator',
       'Has Fireplace', 'Has Garage', 'Has Parking', 'Is New Construction',
       'Is New Construction Potential', 'Is Tenant2Tenant',
       'Is Wheelchairaccessible', 'Jahr', 'Leerwohnungsziffer',
       'Listing Description', 'Listing Title', 'Livingspace',
       'Number Of Rooms', 'Number Of Rooms Cleaned', 'Number of Documents',
       'Number of Images', 'Price Extra Normalized', 'Price Gross Normalized',
       'Price M2 Normalized', 'Price Net Normalized', 'Property Reference Id',
       'Size M2 Normalized', 'Subcategory En Idx', 'Year Built',
       'Year Lastrenovated'],
      dtype='object')

In [ ]:
features = data[['Are Pets Allowed', 'Category Idx', 'Day of Advertisement Created',
       'Day of Date Available From', 'Floor', 'Geo Canton',
       'Geo City', 'Geo Zip', 'Has Balcony', 'Has Cabletv', 'Has Elevator',
       'Has Fireplace', 'Has Garage', 'Has Parking', 'Is New Construction',
       'Is New Construction Potential', 'Is Tenant2Tenant',
       'Is Wheelchairaccessible', 'Jahr', 'Leerwohnungsziffer',
       'Listing Description', 'Listing Title', 'Livingspace',
       'Number Of Rooms', 'Number Of Rooms Cleaned', 'Number of Documents',
       'Number of Images', 'Price Extra Normalized', 'Price Gross Normalized',
       'Price M2 Normalized', 'Price Net Normalized', 'Property Reference Id',
       'Size M2 Normalized', 'Subcategory En Idx', 'Year Built',
       'Year Lastrenovated']]

label = data['Demand']


# Real preprocessing

In [ ]:
def mean_norm(df_input):
    return df_input.apply(lambda x: (x-x.mean())/ x.std(), axis=0)


index = 'Property Reference Id'

def preprocess(df_input):
  binary_features = ['Are Pets Allowed', 'Has Balcony', 'Has Cabletv', 'Has Elevator',
                  'Has Fireplace', 'Has Garage', 'Has Parking', 'Is New Construction',
                  'Is New Construction Potential', 'Is Tenant2Tenant',
                  'Is Wheelchairaccessible']
  for column in binary_features:
    df_input.loc[df_input[column] == False, column] = 0
    df_input.loc[df_input[column] == True, column] = 1

  df_input = pd.get_dummies(df_input, prefix=['Geo'], columns=['Geo Canton'])

  df_input['Year Lastrenovated'].fillna(0, inplace = True)

  threshold=datetime.datetime(2022,9,1)
  df_input['Days Between September and Created']=pd.to_datetime(df_input['Day of Advertisement Created']).apply(lambda x:(threshold-x).days)
  df_input['Days Between September and Available']=pd.to_datetime(df_input['Day of Date Available From']).apply(lambda x:(threshold-x).days)                                                           
  df_input.drop(['Day of Advertisement Created','Day of Date Available From'],axis=1,inplace=True)

  df_input['Leerwohnungsziffer'] = pd.to_numeric(df_input['Leerwohnungsziffer'])
  df_input['Jahr'] = pd.to_numeric(df_input['Jahr'])

  input_cols = df_input[['Are Pets Allowed', 'Floor',
       'Has Balcony', 'Has Cabletv', 'Has Elevator',
       'Has Fireplace', 'Has Garage', 'Has Parking', 'Is New Construction',
       'Is New Construction Potential', 'Is Tenant2Tenant',
       'Is Wheelchairaccessible', 'Livingspace', 'Number Of Rooms', 'Number Of Rooms Cleaned',
       'Number of Documents', 'Number of Images', 'Price Extra Normalized',
       'Price Gross Normalized', 'Price M2 Normalized', 'Price Net Normalized', 'Size M2 Normalized', 
       'Year Lastrenovated', 'Leerwohnungsziffer', 'Jahr',
       'Geo_AG', 'Geo_AR', 'Geo_BE', 'Geo_BL', 'Geo_BS', 'Geo_FR', 'Geo_GE', 
       'Geo_GL', 'Geo_GR', 'Geo_JU', 'Geo_LU', 'Geo_NE', 'Geo_NW', 'Geo_SG', 
       'Geo_SH', 'Geo_SO', 'Geo_SZ', 'Geo_TG', 'Geo_TI', 'Geo_UR', 'Geo_VD', 'Geo_VS', 'Geo_ZG', 'Geo_ZH',
       'Days Between September and Created', 'Days Between September and Available' ]]

  # Normalization of feature columns

  input_cols_norm = mean_norm(input_cols)
  input_cols_norm.fillna(input_cols_norm.median(), inplace=True)

  # input_cols_norm['Day of Advertisement Created'] = df_input['Day of Advertisement Created']
  # input_cols_norm['Day of Date Available From'] = df_input['Day of Date Available From']

  first_column = input_cols_norm.pop('Days Between September and Created')
  input_cols_norm.insert(0, 'Days Between September and Created', first_column)
  second_column = input_cols_norm.pop('Days Between September and Available')
  input_cols_norm.insert(1, 'Days Between September and Available', second_column)

  input_cols_norm['Property Reference Id'] = df_input[index]
  
  return df_input, input_cols_norm


data, input_cols_norm = preprocess(data)
data_test, input_cols_norm_test = preprocess(data_test)

output_cols = list(data['Demand'])
input_cols_norm['Demand'] = data['Demand']
input_cols_norm_test['Demand'] = np.nan


In [ ]:
input_cols_norm

,Days Between September and Created,Days Between September and Available,Are Pets Allowed,Floor,Has Balcony,Has Cabletv,Has Elevator,Has Fireplace,Has Garage,Has Parking,...,Geo_SZ,Geo_TG,Geo_TI,Geo_UR,Geo_VD,Geo_VS,Geo_ZG,Geo_ZH,Property Reference Id,Demand
0,-0.849392,-0.796992,-0.832208,-1.067466,-0.200172,0.758779,0.798902,-0.156072,0.899897,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,1.382123,0002.03.7648,25.0
1,-0.151621,-0.189807,-0.832208,-0.602958,-0.200172,0.758779,0.798902,-0.156072,0.899897,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.10473,-0.723488,9680.61.2071,2.0
2,1.488814,-0.038427,-0.832208,-0.602958,-0.200172,-1.317803,-1.251618,-0.156072,-1.111143,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,7204.61.8658,0.0
3,-1.152990,-0.898467,1.201510,-0.602958,-0.200172,0.758779,0.798902,-0.156072,0.899897,1.090902,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,7410.50.5987,4.0
4,1.097994,1.378893,-0.832208,-1.067466,-0.200172,-1.317803,-1.251618,-0.156072,-1.111143,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,9270.61.7822,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.994000,0.974658,-0.832208,-0.138449,-0.200172,-1.317803,-1.251618,-0.156072,-1.111143,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,4458.50.0065,2.0
19996,-1.352593,-1.357599,1.201510,-0.138449,-0.200172,0.758779,0.798902,-0.156072,0.899897,1.090902,...,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.10473,-0.723488,4481.61.5904,1.0
19997,-1.320724,-0.974989,-0.832208,1.255078,-0.200172,0.758779,0.798902,-0.156072,0.899897,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,0221.61.7685,2.0
19998,-0.685014,-0.898467,-0.832208,2.184095,-0.200172,0.758779,0.798902,-0.156072,0.899897,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.10473,-0.723488,8508.50.4399,2.0


In [ ]:
input_cols_norm_test[ input_cols_norm_test['Property Reference Id'] == "8341.61.6570" ]


,Days Between September and Created,Days Between September and Available,Are Pets Allowed,Floor,Has Balcony,Has Cabletv,Has Elevator,Has Fireplace,Has Garage,Has Parking,...,Geo_SZ,Geo_TG,Geo_TI,Geo_UR,Geo_VD,Geo_VS,Geo_ZG,Geo_ZH,Property Reference Id,Demand
6862,-1.311274,-1.011595,-0.831687,2.701784,-0.212815,0.767208,0.812694,-0.160562,0.905138,1.128438,...,-0.108799,-0.125057,-0.154785,-0.043628,-0.255532,-0.093133,-0.106422,-0.734569,8341.61.6570,NaN


In [ ]:
input_cols_norm.head(5)

,Days Between September and Created,Days Between September and Available,Are Pets Allowed,Floor,Has Balcony,Has Cabletv,Has Elevator,Has Fireplace,Has Garage,Has Parking,...,Geo_SZ,Geo_TG,Geo_TI,Geo_UR,Geo_VD,Geo_VS,Geo_ZG,Geo_ZH,Property Reference Id,Demand
0,-0.849392,-0.796992,-0.832208,-1.067466,-0.200172,0.758779,0.798902,-0.156072,0.899897,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,1.382123,0002.03.7648,25.0
1,-0.151621,-0.189807,-0.832208,-0.602958,-0.200172,0.758779,0.798902,-0.156072,0.899897,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.10473,-0.723488,9680.61.2071,2.0
2,1.488814,-0.038427,-0.832208,-0.602958,-0.200172,-1.317803,-1.251618,-0.156072,-1.111143,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,7204.61.8658,0.0
3,-1.152990,-0.898467,1.201510,-0.602958,-0.200172,0.758779,0.798902,-0.156072,0.899897,1.090902,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,7410.50.5987,4.0
4,1.097994,1.378893,-0.832208,-1.067466,-0.200172,-1.317803,-1.251618,-0.156072,-1.111143,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,9270.61.7822,3.0


# Merge with image dataset



In [ ]:
images_data_train.columns

Index(['Property Reference Id', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       '9', '10', '11', '12', '13', '14', '15', 'Demand'],
      dtype='object')

In [ ]:
images_data_test.columns

Index(['Property Reference Id', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       '9', '10', '11', '12', '13', '14', '15'],
      dtype='object')

In [ ]:
def merge_dataset(tabular_data, images_data):

  images_id = images_data['Property Reference Id'].unique()
  # print("image_id:", images_id)
  # print("len(images_id):", len(images_id))

  tabular_data = tabular_data[tabular_data['Property Reference Id'].isin(images_id)]

  images_data['Property Reference Id'] = images_data['Property Reference Id'].astype('category')
  tabular_data['Property Reference Id'] = tabular_data['Property Reference Id'].astype('category')

  images_data.drop('Demand', inplace=True, axis=1)
  data_joined = images_data.merge(tabular_data, on='Property Reference Id', how='left')

  return data_joined

def merge_dataset_test(tabular_data, images_data):

  images_id = images_data['Property Reference Id'].unique()
  # print("image_id:", images_id)
  # print("len(images_id):", len(images_id))

  tabular_data = tabular_data[tabular_data['Property Reference Id'].isin(images_id)]

  images_data['Property Reference Id'] = images_data['Property Reference Id'].astype('category')
  tabular_data['Property Reference Id'] = tabular_data['Property Reference Id'].astype('category')

  data_joined = images_data.merge(tabular_data, on='Property Reference Id', how='left')

  return data_joined


data_joined = merge_dataset(input_cols_norm, images_data_train)
data_joined_test = merge_dataset_test(input_cols_norm_test, images_data_test)


<ipython-input-304-3e2578e8a086>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabular_data['Property Reference Id'] = tabular_data['Property Reference Id'].astype('category')
<ipython-input-304-3e2578e8a086>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabular_data['Property Reference Id'] = tabular_data['Property Reference Id'].astype('category')


In [ ]:
# data_joined.drop('Property Reference Id', inplace=True, axis=1)
data_joined['Demand']

0        2.0
1        2.0
2        2.0
3       17.0
4       17.0
        ... 
8476     5.0
8477     5.0
8478     5.0
8479     5.0
8480     5.0
Name: Demand, Length: 8481, dtype: float64

In [ ]:
data_joined.columns
data_joined_test.columns

Index(['Property Reference Id', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       '9', '10', '11', '12', '13', '14', '15',
       'Days Between September and Created',
       'Days Between September and Available', 'Are Pets Allowed', 'Floor',
       'Has Balcony', 'Has Cabletv', 'Has Elevator', 'Has Fireplace',
       'Has Garage', 'Has Parking', 'Is New Construction',
       'Is New Construction Potential', 'Is Tenant2Tenant',
       'Is Wheelchairaccessible', 'Livingspace', 'Number Of Rooms',
       'Number Of Rooms Cleaned', 'Number of Documents', 'Number of Images',
       'Price Extra Normalized', 'Price Gross Normalized',
       'Price M2 Normalized', 'Price Net Normalized', 'Size M2 Normalized',
       'Year Lastrenovated', 'Leerwohnungsziffer', 'Jahr', 'Geo_AG', 'Geo_AR',
       'Geo_BE', 'Geo_BL', 'Geo_BS', 'Geo_FR', 'Geo_GE', 'Geo_GL', 'Geo_GR',
       'Geo_JU', 'Geo_LU', 'Geo_NE', 'Geo_NW', 'Geo_SG', 'Geo_SH', 'Geo_SO',
       'Geo_SZ', 'Geo_TG', 'Geo_TI', 'Geo_UR', 'Geo_

In [ ]:
data_joined

,Property Reference Id,0,1,2,3,4,5,6,7,8,...,Geo_SO,Geo_SZ,Geo_TG,Geo_TI,Geo_UR,Geo_VD,Geo_VS,Geo_ZG,Geo_ZH,Demand
0,0002.61.6915,-0.221832,6.024265,5.953249,-6.890490,3.063627,-0.948176,-0.362961,-1.883066,0.872796,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,1.382123,2.0
1,0002.61.6915,0.609485,6.242800,5.783226,-6.939771,3.479691,-0.874210,0.335041,-1.675094,0.111615,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,1.382123,2.0
2,0002.61.6915,1.398565,6.613178,6.145263,-7.383204,3.698415,-2.127483,0.909472,-2.784360,-0.298863,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,1.382123,2.0
3,0012.61.9682,0.732772,7.525365,6.753076,-8.926143,5.028864,-3.052044,0.783926,-4.661302,0.409809,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.10473,-0.723488,17.0
4,0012.61.9682,-1.362888,5.466207,5.663973,-6.751781,2.632468,-2.109637,-0.573149,-2.908666,1.750561,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.10473,-0.723488,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8476,9995.50.8394,2.686135,10.535210,8.687286,-11.613550,6.980347,-1.652806,1.780055,0.754969,-1.321127,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,5.0
8477,9995.50.8394,0.648810,8.603827,7.652774,-9.628755,4.941155,0.387756,0.320485,1.968812,-0.037644,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,5.0
8478,9995.50.8394,1.953830,9.546097,7.472683,-9.778398,7.437314,-3.499745,4.520870,8.898790,-1.808473,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,5.0
8479,9995.50.8394,-1.864435,7.370160,6.550863,-8.974305,5.658174,-1.539836,-0.353800,3.408514,2.730169,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,5.0


## merge version 1

In [ ]:
# images_data = pd.read_csv(PATH + "images_vectorized.csv")
images_data = pd.read_csv(PATH + "images_vectorized_2.csv")

images_id = images_data['Property Reference Id'].unique()
print(images_id)
print(len(images_id))


['0002.61.6915' '0012.61.9682' '0043.61.2130' '0051.06.1645'
 '0053.61.6292' '0092.22.2130' '0092.26.3101' '0129.26.1770'
 '0129.61.9412' '0200.26.4088' '0214.61.8006' '0221.61.7685'
 '0254.50.1151' '0286.26.1334' '0286.50.7822' '0292.61.3115'
 '0294.61.7107' '0300.61.6054' '0300.61.9894' '0311.50.7705'
 '0311.61.1334' '0356.50.7822' '0359.61.2130' '0480.06.4593'
 '0618.06.0031' '0662.26.4741' '0662.26.9662' '0662.50.9670'
 '0668.61.0065' '0759.61.3115' '0759.61.6450' '0789.34.5829'
 '0789.34.7688' '0836.50.1151' '0837.50.0065' '0862.06.3115'
 '0908.26.5664' '0913.22.3690' '0947.61.9754' '0947.92.2532'
 '0960.61.2321' '1036.50.0065' '1036.50.6369' '1036.50.8115'
 '1036.61.6915' '1036.61.7416' '1036.61.7543' '1038.06.5834'
 '1038.50.0065' '1073.26.9682' '1076.06.4741' '1076.06.9424'
 '1076.06.9662' '1076.50.2130' '1107.61.9424' '1120.26.1351'
 '1137.26.1334' '1137.61.5386' '1232.61.9935' '1297.50.0031'
 '1297.50.7822' '1297.50.9894' '1297.61.5386' '1297.61.9754'
 '1308.50.5240' '1308.50

In [ ]:
input_cols_norm_reduced = input_cols_norm[input_cols_norm['Property Reference Id'].isin(images_id)]
input_cols_norm_reduced

,Day of Date Available From,Day of Advertisement Created,Are Pets Allowed,Floor,Has Balcony,Has Cabletv,Has Elevator,Has Fireplace,Has Garage,Has Parking,...,Geo_SZ,Geo_TG,Geo_TI,Geo_UR,Geo_VD,Geo_VS,Geo_ZG,Geo_ZH,Demand,Property Reference Id
51,"September 1, 2020","July 3, 2020",-0.832208,0.326060,-0.200172,0.758779,0.798902,-0.156072,0.899897,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.104730,-0.723488,19.0,7415.50.2130
56,"September 1, 2018","August 6, 2018",-0.832208,-1.067466,-0.200172,-1.317803,0.798902,-0.156072,-1.111143,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,9.525680,-0.104730,-0.723488,0.0,5469.26.4284
140,"March 1, 2019","October 24, 2018",-0.832208,-0.138449,-0.200172,-1.317803,-1.251618,-0.156072,-1.111143,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.104730,-0.723488,0.0,7276.22.0065
145,"March 1, 2022","November 30, 2021",-0.832208,0.326060,-0.200172,0.758779,0.798902,-0.156072,0.899897,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.104730,-0.723488,8.0,5361.50.7934
177,"November 1, 2018","August 7, 2018",1.201510,-0.602958,-0.200172,0.758779,-1.251618,-0.156072,-1.111143,1.090902,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.104730,-0.723488,0.0,4175.61.4362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19888,"September 1, 2017","June 2, 2017",-0.832208,0.326060,-0.200172,-1.317803,-1.251618,-0.156072,-1.111143,-0.916588,...,-0.097148,-0.132288,-0.147698,19.585612,-0.259622,-0.104974,-0.104730,-0.723488,0.0,8396.61.5664
19932,"May 1, 2021","April 15, 2021",-0.832208,-0.138449,-0.200172,0.758779,0.798902,-0.156072,0.899897,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.104730,-0.723488,2.0,0092.26.3101
19941,"April 1, 2020","March 4, 2020",-0.832208,-0.602958,-0.200172,0.758779,0.798902,-0.156072,0.899897,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.104730,1.382123,16.0,5738.50.4362
19976,"April 1, 2019","January 17, 2019",-0.832208,-1.067466,-0.200172,0.758779,0.798902,-0.156072,0.899897,-0.916588,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,9.547845,-0.723488,13.0,8994.68.7822


In [ ]:
# images_data = pd.read_csv(PATH + "images_vectorized.csv")
images_data = pd.read_csv(PATH + "images_vectorized_2.csv")

images_data['Property Reference Id'] = images_data['Property Reference Id'].astype('category')
input_cols_norm_reduced['Property Reference Id'] = input_cols_norm_reduced['Property Reference Id'].astype('category')
images_data

<ipython-input-24-3d3e7fa5b4a0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_cols_norm_reduced['Property Reference Id'] = input_cols_norm_reduced['Property Reference Id'].astype('category')


,Property Reference Id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,Demand
0,0002.61.6915,-0.221832,6.024265,5.953249,-6.890490,3.063627,-0.948176,-0.362961,-1.883066,0.872796,6.887189,-9.850289,-0.950801,-2.359023,-7.314687,9.682235,-2.985931,2.0
1,0002.61.6915,0.609485,6.242800,5.783226,-6.939771,3.479691,-0.874210,0.335041,-1.675094,0.111615,6.779986,-9.572109,-0.092465,-2.760837,-7.171674,9.206720,-2.622147,2.0
2,0002.61.6915,1.398565,6.613178,6.145263,-7.383204,3.698415,-2.127483,0.909472,-2.784360,-0.298863,7.241723,-10.057664,0.637905,-3.071800,-7.520823,9.508305,-3.589651,2.0
3,0012.61.9682,0.732772,7.525365,6.753076,-8.926143,5.028864,-3.052044,0.783926,-4.661302,0.409809,7.880892,-12.079361,0.409625,-3.946983,-8.715199,11.561431,-4.128768,17.0
4,0012.61.9682,-1.362888,5.466207,5.663973,-6.751781,2.632468,-2.109637,-0.573149,-2.908666,1.750561,6.727478,-9.977320,-1.542024,-1.799264,-7.152509,10.005502,-2.396945,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5838,9995.50.8394,2.686135,10.535210,8.687286,-11.613550,6.980347,-1.652806,1.780055,0.754969,-1.321127,10.430278,-15.639960,2.439432,-6.165233,-11.678176,13.718158,-2.538267,5.0
5839,9995.50.8394,0.648810,8.603827,7.652774,-9.628755,4.941155,0.387756,0.320485,1.968812,-0.037644,9.185239,-13.673468,0.599522,-4.164209,-10.169299,12.084097,-1.126678,5.0
5840,9995.50.8394,1.953830,9.546097,7.472683,-9.778398,7.437314,-3.499745,4.520870,8.898790,-1.808473,8.530967,-13.685159,3.585144,-6.815229,-10.193208,12.663684,7.011076,5.0
5841,9995.50.8394,-1.864435,7.370160,6.550863,-8.974305,5.658174,-1.539836,-0.353800,3.408514,2.730169,7.720353,-13.286383,-1.674972,-4.154854,-8.953053,13.140576,2.727453,5.0


In [ ]:
images_data.drop('Demand', inplace=True, axis=1)
data_joined = images_data.merge(input_cols_norm_reduced, on='Property Reference Id', how='left')
data_joined

,Property Reference Id,0,1,2,3,4,5,6,7,8,...,Geo_SO,Geo_SZ,Geo_TG,Geo_TI,Geo_UR,Geo_VD,Geo_VS,Geo_ZG,Geo_ZH,Demand
0,0002.61.6915,-0.221832,6.024265,5.953249,-6.890490,3.063627,-0.948176,-0.362961,-1.883066,0.872796,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,1.382123,2.0
1,0002.61.6915,0.609485,6.242800,5.783226,-6.939771,3.479691,-0.874210,0.335041,-1.675094,0.111615,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,1.382123,2.0
2,0002.61.6915,1.398565,6.613178,6.145263,-7.383204,3.698415,-2.127483,0.909472,-2.784360,-0.298863,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,1.382123,2.0
3,0012.61.9682,0.732772,7.525365,6.753076,-8.926143,5.028864,-3.052044,0.783926,-4.661302,0.409809,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.10473,-0.723488,17.0
4,0012.61.9682,-1.362888,5.466207,5.663973,-6.751781,2.632468,-2.109637,-0.573149,-2.908666,1.750561,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.10473,-0.723488,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8476,9995.50.8394,2.686135,10.535210,8.687286,-11.613550,6.980347,-1.652806,1.780055,0.754969,-1.321127,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,5.0
8477,9995.50.8394,0.648810,8.603827,7.652774,-9.628755,4.941155,0.387756,0.320485,1.968812,-0.037644,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,5.0
8478,9995.50.8394,1.953830,9.546097,7.472683,-9.778398,7.437314,-3.499745,4.520870,8.898790,-1.808473,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,5.0
8479,9995.50.8394,-1.864435,7.370160,6.550863,-8.974305,5.658174,-1.539836,-0.353800,3.408514,2.730169,...,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,5.0


# Split dataset into training and validation set

In [ ]:
import random
import math

# 1) Case we test our model on validation set ---------------------------------------------------------------------------------------------------------------------------------
unique_ids=data_joined['Property Reference Id'].unique()

# We define the ID we use for training and those we use for testing
ids_train=np.array(random.sample(set(unique_ids), math.floor(0.8*len(unique_ids))))
ids_test=np.array([x for x in unique_ids if x not in ids_train])

#We define the train dataset and the test dataset
data_joined_train=data_joined[data_joined['Property Reference Id'].isin(ids_train)]
data_joined_test=data_joined[data_joined['Property Reference Id'].isin(ids_test)]

print(len(unique_ids),len(ids_train),len(ids_test))



# # 2) Case for Harald predictions on test set --------------------------------------------------------------------------------------------------------------------------------

# unique_ids=data_joined['Property Reference Id'].unique()
# unique_ids_test=data_joined_test['Property Reference Id'].unique()


# # We define the ID we use for training and those we use for testing
# ids_train=np.array(random.sample(set(unique_ids), math.floor(1*len(unique_ids))))
# # ids_test=np.array(random.sample(set(unique_ids), math.floor(1*len(unique_ids_test))))

# #We define the train dataset and the test dataset
# data_joined_train=data_joined[data_joined['Property Reference Id'].isin(ids_train)]
# # data_joined_test=data_joined[data_joined['Property Reference Id'].isin(ids_test)]

# print(len(data_joined_train),len(data_joined_test))


686 548 138


# Data to tensors

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.autograd import Variable
import torchvision.models as models

In [ ]:
#TRAIN PART ------------------------------------------------------------------------------------------------------------------------------------

label = data_joined_train['Demand']
output_cols = list(label)

input_cols = data_joined_train[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', 'Are Pets Allowed', 'Floor', 'Has Balcony',
       'Has Cabletv', 'Has Elevator', 'Has Fireplace', 'Has Garage',
       'Has Parking', 'Is New Construction', 'Is New Construction Potential',
       'Is Tenant2Tenant', 'Is Wheelchairaccessible', 'Livingspace',
       'Number Of Rooms', 'Number Of Rooms Cleaned', 'Number of Documents',
       'Number of Images', 'Price Extra Normalized', 'Price Gross Normalized',
       'Price M2 Normalized', 'Price Net Normalized', 'Size M2 Normalized',
       'Year Lastrenovated', 'Leerwohnungsziffer', 'Jahr', 'Geo_AG', 'Geo_AR',
       'Geo_BE', 'Geo_BL', 'Geo_BS', 'Geo_FR', 'Geo_GE', 'Geo_GL', 'Geo_GR',
       'Geo_JU', 'Geo_LU', 'Geo_NE', 'Geo_NW', 'Geo_SG', 'Geo_SH', 'Geo_SO',
       'Geo_SZ', 'Geo_TG', 'Geo_TI', 'Geo_UR', 'Geo_VD', 'Geo_VS', 'Geo_ZG',
       'Geo_ZH']]


#TEST PART -----------------------------------------------------------------------------------------------------------------------------------

label_test = data_joined_test['Demand']
output_cols_test = list(label_test)

input_cols_test = data_joined_test[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', 'Are Pets Allowed', 'Floor', 'Has Balcony',
       'Has Cabletv', 'Has Elevator', 'Has Fireplace', 'Has Garage',
       'Has Parking', 'Is New Construction', 'Is New Construction Potential',
       'Is Tenant2Tenant', 'Is Wheelchairaccessible', 'Livingspace',
       'Number Of Rooms', 'Number Of Rooms Cleaned', 'Number of Documents',
       'Number of Images', 'Price Extra Normalized', 'Price Gross Normalized',
       'Price M2 Normalized', 'Price Net Normalized', 'Size M2 Normalized',
       'Year Lastrenovated', 'Leerwohnungsziffer', 'Jahr', 'Geo_AG', 'Geo_AR',
       'Geo_BE', 'Geo_BL', 'Geo_BS', 'Geo_FR', 'Geo_GE', 'Geo_GL', 'Geo_GR',
       'Geo_JU', 'Geo_LU', 'Geo_NE', 'Geo_NW', 'Geo_SG', 'Geo_SH', 'Geo_SO',
       'Geo_SZ', 'Geo_TG', 'Geo_TI', 'Geo_UR', 'Geo_VD', 'Geo_VS', 'Geo_ZG',
       'Geo_ZH']]


In [ ]:
input_cols

,0,1,2,3,4,5,6,7,8,9,...,Geo_SH,Geo_SO,Geo_SZ,Geo_TG,Geo_TI,Geo_UR,Geo_VD,Geo_VS,Geo_ZG,Geo_ZH
0,-0.221832,6.024265,5.953249,-6.890490,3.063627,-0.948176,-0.362961,-1.883066,0.872796,6.887189,...,-0.099737,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,1.382123
1,0.609485,6.242800,5.783226,-6.939771,3.479691,-0.874210,0.335041,-1.675094,0.111615,6.779986,...,-0.099737,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,1.382123
2,1.398565,6.613178,6.145263,-7.383204,3.698415,-2.127483,0.909472,-2.784360,-0.298863,7.241723,...,-0.099737,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,1.382123
33,0.068654,5.019561,5.034229,-5.608108,2.298580,-0.579629,0.511757,1.390740,0.383952,5.980761,...,-0.099737,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488
34,0.191669,4.391746,4.526442,-4.894406,1.624116,-0.187547,0.791065,2.323790,0.485652,5.463934,...,-0.099737,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8476,2.686135,10.535210,8.687286,-11.613550,6.980347,-1.652806,1.780055,0.754969,-1.321127,10.430278,...,-0.099737,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488
8477,0.648810,8.603827,7.652774,-9.628755,4.941155,0.387756,0.320485,1.968812,-0.037644,9.185239,...,-0.099737,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488
8478,1.953830,9.546097,7.472683,-9.778398,7.437314,-3.499745,4.520870,8.898790,-1.808473,8.530967,...,-0.099737,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488
8479,-1.864435,7.370160,6.550863,-8.974305,5.658174,-1.539836,-0.353800,3.408514,2.730169,7.720353,...,-0.099737,-0.196833,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488


In [ ]:
#TRAIN PART ------------------------------------------------------------------------------------------------------------------------------------

inputs_array, targets_array1 = np.array(input_cols.to_numpy()), label.to_numpy()
targets_array=[]
#We add brakets for tensor structure... (can be done differently)
for elt in targets_array1:
    targets_array.append([elt])
targets_array = np.array(targets_array)

print(type(inputs_array))
print(type(targets_array))
print(inputs_array)
print(targets_array)

inputs = torch.from_numpy(inputs_array).to(torch.float32)
targets = torch.from_numpy(targets_array).to(torch.float32)
dataset = TensorDataset(inputs, targets)

print("train dataset:", dataset)


#TEST PART -----------------------------------------------------------------------------------------------------------------------------------


inputs_array_test, targets_array1_test = np.array(input_cols_test.to_numpy()), label_test.to_numpy()
targets_array_test=[]
#We add brakets for tensor structure... (can be done differently)
for elt in targets_array1_test:
    targets_array_test.append([elt])
targets_array_test = np.array(targets_array_test)

print(type(inputs_array_test))
print(type(targets_array_test))
print(inputs_array_test)
print(targets_array_test)

inputs_test = torch.from_numpy(inputs_array_test).to(torch.float32)
targets_test = torch.from_numpy(targets_array_test).to(torch.float32)
dataset_test = TensorDataset(inputs_test, targets_test)

print("test dataset:", dataset_test)




<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[[-0.22183222  6.0242653   5.953249   ... -0.10497413 -0.10473044
   1.3821235 ]
 [ 0.60948485  6.2427998   5.7832255  ... -0.10497413 -0.10473044
   1.3821235 ]
 [ 1.3985655   6.613178    6.1452627  ... -0.10497413 -0.10473044
   1.3821235 ]
 ...
 [ 1.9538296   9.546097    7.4726825  ... -0.10497413 -0.10473044
  -0.72348817]
 [-1.8644348   7.3701596   6.550863   ... -0.10497413 -0.10473044
  -0.72348817]
 [ 0.11926779  6.666862    5.9892855  ... -0.10497413 -0.10473044
  -0.72348817]]
[[2.]
 [2.]
 [2.]
 ...
 [5.]
 [5.]
 [5.]]
train dataset: <torch.utils.data.dataset.TensorDataset object at 0x7f5e8b3bd9a0>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[[ 0.73277164  7.525365    6.753076   ... -0.10497413 -0.10473044
  -0.72348817]
 [-1.3628877   5.466207    5.663973   ... -0.10497413 -0.10473044
  -0.72348817]
 [-1.3504109   5.82759     5.4475074  ... -0.10497413 -0.10473044
  -0.72348817]
 ...
 [-0.94495565  3.893086    4.3687196  ...

In [ ]:
# VAL_PERCENT = 0.2
# NUM_ROWS = int(len(data_joined))
# val_size = int(NUM_ROWS * VAL_PERCENT)
# train_size = NUM_ROWS - val_size

# train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_ds = dataset
val_ds = dataset_test
test_ds = dataset_pred

print(len(train_ds), len(val_ds))

6726 1755


In [ ]:
BATCH_SIZE = 128

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)

for xb, yb, in train_loader:
    print("inputs:", xb)
    print("targets:", yb)
    break

inputs: tensor([[ 1.6409,  5.5675,  5.2100,  ..., -0.1050, -0.1047, -0.7235],
        [ 1.5594,  7.5466,  6.6712,  ..., -0.1050,  9.5478, -0.7235],
        [ 1.2719,  4.7390,  4.5937,  ..., -0.1050, -0.1047, -0.7235],
        ...,
        [ 1.0530,  5.5495,  5.2144,  ..., -0.1050, -0.1047, -0.7235],
        [ 1.4952,  9.5329,  7.8219,  ..., -0.1050, -0.1047,  1.3821],
        [ 0.5367,  5.1076,  5.0733,  ..., -0.1050, -0.1047, -0.7235]])
targets: tensor([[ 1.],
        [14.],
        [ 0.],
        [ 1.],
        [ 4.],
        [ 0.],
        [ 3.],
        [ 1.],
        [ 2.],
        [ 1.],
        [14.],
        [ 3.],
        [ 7.],
        [ 4.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 1.],
        [ 1.],
        [ 8.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 4.],
        [ 1.],
        [ 2.],
        [ 0.],
        [ 0.],
        [18.],
        [ 1.],
        [ 3.],
        [ 4.],
        [ 8.],
        [ 0.],
        [ 1.],
        [

# Model


In [ ]:
#Creation of linear regression model

in_cols = input_cols.columns
out_cols = ['Demand']

input_size = len(in_cols)
output_size = len(out_cols)

class PredictModel(nn.Module):


    def __init__(self):
        # super().__init__()
        # self.linear = nn.Linear(input_size, output_size)
        super(PredictModel, self).__init__()
        self.layer_1 = nn.Linear(input_size, 32)
        self.layer_2 = nn.Linear(32, 64)
        self.layer_3 = nn.Linear(64, 128)
        # self.layer_4 = nn.Linear(64, 128)
        # self.layer_5 = nn.Linear(128, 64)
        self.layer_4 = nn.Linear(128, 64)
        self.layer_5 = nn.Linear(64, 32)
        self.layer_out = nn.Linear(32, output_size)
        
        self.relu = nn.ReLU()
    
    def forward(self, xb):
        # out = self.linear(xb)
        x = self.relu(self.layer_1(xb))
        x = self.relu(self.layer_2(x))
        x = self.relu(self.layer_3(x))
        x = self.relu(self.layer_4(x))
        x = self.relu(self.layer_5(x))
        # x = self.relu(self.layer_6(x))
        # x = self.relu(self.layer_7(x))
        x = self.layer_out(x)
        return x


    def training_step(self, batch):
        inputs, targets = batch
        out = self(inputs)
        loss = F.l1_loss(out, targets)
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        out = self(inputs)
        loss = F.l1_loss(out, targets)
        return {'val_loss': loss.detach()}
    
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        if (epoch+1) % 20 == 0 or epoch == num_epochs - 1:
            print("Epoch[{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

In [ ]:
#Defining our model 
model = PredictModel()
list(model.parameters())

#Training loop
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history=[]
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

result = evaluate(model, val_loader)
print(result)




{'val_loss': 3.8297526836395264}


In [ ]:
epochs = 300
lr = 0.001
history = fit(epochs, lr, model, train_loader, val_loader)

Epoch[20], val_loss: 3.3626
Epoch[40], val_loss: 3.3768
Epoch[60], val_loss: 3.3601
Epoch[80], val_loss: 3.3396
Epoch[100], val_loss: 3.2916
Epoch[120], val_loss: 3.1876
Epoch[140], val_loss: 3.1560
Epoch[160], val_loss: 2.9818
Epoch[180], val_loss: 3.0018
Epoch[200], val_loss: 2.9023
Epoch[220], val_loss: 2.9416
Epoch[240], val_loss: 3.0160
Epoch[260], val_loss: 2.9281
Epoch[280], val_loss: 2.9949
Epoch[300], val_loss: 2.8482


In [ ]:
# We try to test our model on a random value of the test set

def predict_single(input, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)         
    prediction = predictions[0].detach()
    return prediction

input, target = val_ds[50]
prediction = predict_single(input, model)

print("Input:", input)
print("Target:", target)
print("Prediction:", prediction)
print()


# Return the entire predictions (Tensor Y_label)

def predict_all(val_ds, model):
    predictions = []
    targets = []
    for i in range(len(val_ds)):
       input, target = val_ds[i]
       prediction = predict_single(input, model)
       predictions.append(prediction.numpy()[0])
       targets.append(target)
    return predictions, targets

predictions, targets = predict_all(val_ds, model)

#On arrondi à l'entier le plus proche pour effectuer notre prédiction et on remplace les valeurs négatives

predictions = [max(10**(-0.05),k) for k in predictions]



Input: tensor([ 2.0991,  6.5296,  5.8832, -7.4918,  4.1345, -1.8794,  2.2303,  2.1299,
        -0.6852,  7.0435, -9.7505,  1.4381, -3.4204, -7.5194,  8.5891, -0.7026,
        -0.8322, -1.0675, -0.2002,  0.7588,  0.7989, -0.1561,  0.8999, -0.9166,
        -0.0794, -0.1257, -0.1090, -0.0535, -1.6130, -1.2049, -1.2049, -1.4841,
         1.5834, -0.7723, -0.7328,  0.0682, -0.6882, -1.6134, -0.9030, -0.7763,
         1.6861, -0.3558, -0.0490, -0.2556, -0.2370, -0.1906, -0.1600, -0.2108,
        -0.0436, -0.0723, -0.0122, -0.2217, -0.1367, -0.0464, -0.2688, -0.0997,
        -0.1968, -0.0971, -0.1323, -0.1477, -0.0511,  3.8516, -0.1050, -0.1047,
        -0.7235])
Target: tensor([3.])
Prediction: tensor([3.6799])



In [ ]:
print(predictions)

[0.8912509381337456, 0.8912509381337456, 11.951239, 12.862627, 15.712616, 16.296003, 12.629386, 13.323881, 13.058647, 13.761873, 11.821325, 12.539836, 12.963602, 13.8610525, 14.155253, 14.749472, 11.8868265, 12.580972, 12.421556, 13.256086, 14.441715, 15.023104, 11.502541, 12.197082, 12.113951, 12.7645, 12.564449, 13.25085, 12.341272, 13.043658, 4.789196, 4.846039, 4.5992837, 4.524922, 4.4307737, 4.377219, 5.103767, 5.0122423, 4.4252124, 4.3223305, 4.789892, 4.7584004, 4.2447686, 4.3309455, 5.213869, 5.1195693, 4.335188, 4.4261765, 4.0015216, 4.0752773, 3.679859, 3.750419, 4.5811124, 4.653085, 4.3875127, 4.4721885, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.891250938133745

In [ ]:
#Computation of the accurcy of our model on test set
EPSILON = 2

def accuracy(val_ds, model, EPSILON):
    corrects = 0
    for i in range(len(val_ds)):
        input, target = val_ds[i]
        prediction = predict_single(input, model)
        prediction = np.round(prediction)
        if prediction <= target + EPSILON and prediction >= target - EPSILON:
            corrects+=1
    accuracy = corrects/len(val_ds)
    print("Accuracy with EPSILON={}:".format(EPSILON), accuracy)
    return accuracy

accuracy(val_ds, model, EPSILON)



#Using Mean Poisson deviance
from sklearn.metrics import mean_poisson_deviance

print("Mean Poisson deviance:", mean_poisson_deviance(targets, predictions))




Accuracy with EPSILON=2: 0.6991452991452991
Mean Poisson deviance: 4.439334759797035


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:746: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  array = np.asarray(array, order=order, dtype=dtype)


# We groupby images by ID

In [ ]:
def groupbyID(data_joined_test):

  # We add the prediction on the test set to gather our results order by id
  data_joined_test['Prediction'] = predictions

  data_joined_test["Property Reference Id"] = data_joined_test["Property Reference Id"].astype(object)

  temp_dataset = data_joined_test.groupby(['Property Reference Id'])
  results = temp_dataset.mean()

  data_joined_test.loc[data_joined_test.Prediction<=0,'Prediction']=10**(-0.05)

  return results

results = groupbyID(data_joined_test)

print()
print()
print("Final mean Poisson deviance(per ID):", mean_poisson_deviance(results["Demand"], results["Prediction"]))







Final mean Poisson deviance(per ID): 5.110922386591531


<ipython-input-321-d04d04e1b3f5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_joined_test['Prediction'] = predictions
<ipython-input-321-d04d04e1b3f5>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_joined_test["Property Reference Id"] = data_joined_test["Property Reference Id"].astype(object)


In [ ]:
results.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', 'Days Between September and Created',
       'Days Between September and Available', 'Are Pets Allowed', 'Floor',
       'Has Balcony', 'Has Cabletv', 'Has Elevator', 'Has Fireplace',
       'Has Garage', 'Has Parking', 'Is New Construction',
       'Is New Construction Potential', 'Is Tenant2Tenant',
       'Is Wheelchairaccessible', 'Livingspace', 'Number Of Rooms',
       'Number Of Rooms Cleaned', 'Number of Documents', 'Number of Images',
       'Price Extra Normalized', 'Price Gross Normalized',
       'Price M2 Normalized', 'Price Net Normalized', 'Size M2 Normalized',
       'Year Lastrenovated', 'Leerwohnungsziffer', 'Jahr', 'Geo_AG', 'Geo_AR',
       'Geo_BE', 'Geo_BL', 'Geo_BS', 'Geo_FR', 'Geo_GE', 'Geo_GL', 'Geo_GR',
       'Geo_JU', 'Geo_LU', 'Geo_NE', 'Geo_NW', 'Geo_SG', 'Geo_SH', 'Geo_SO',
       'Geo_SZ', 'Geo_TG', 'Geo_TI', 'Geo_UR', 'Geo_VD', 'Geo_VS', 'Geo_ZG',
       

# Save and load the model

In [ ]:
#We save the model
torch.save(model, PATH + 'model.pkl')
print("Saved PyTorch Model State to model_weights.pth")

Saved PyTorch Model State to model_weights.pth


In [ ]:
# #We load the model
model_saved = torch.load(PATH + 'model.pkl')
model_saved.eval()
model_saved

PredictModel(
  (layer_1): Linear(in_features=65, out_features=32, bias=True)
  (layer_2): Linear(in_features=32, out_features=64, bias=True)
  (layer_3): Linear(in_features=64, out_features=128, bias=True)
  (layer_4): Linear(in_features=128, out_features=64, bias=True)
  (layer_5): Linear(in_features=64, out_features=32, bias=True)
  (layer_out): Linear(in_features=32, out_features=1, bias=True)
  (relu): ReLU()
)

In [ ]:
model_saved()

## 1st version of groupby

In [ ]:
# We add the prediction on the test set to gather our results order by id
data_joined_test['Prediction'] = predictions
data_joined_test.sample(5)


<ipython-input-75-b8612112f83e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_joined_test['Prediction'] = predictions


,Property Reference Id,0,1,2,3,4,5,6,7,8,...,Geo_SZ,Geo_TG,Geo_TI,Geo_UR,Geo_VD,Geo_VS,Geo_ZG,Geo_ZH,Demand,Prediction
6787,7684.61.7705,-0.502878,5.947612,5.516549,-6.756667,3.567677,-2.863783,1.876810,4.323555,1.001609,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,1.0,0.891251
446,0668.61.0065,-0.463543,5.042772,5.045663,-6.140898,2.529379,-2.926522,1.945866,4.458686,1.526753,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,14.0,1.026594
7252,8341.50.7705,-0.955973,4.044521,4.508246,-4.677773,1.006604,0.447340,0.144032,2.436692,1.052145,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,14.0,1.254935
2125,2490.22.0964,-1.118535,4.259557,4.679176,-4.985484,1.229805,0.267612,-0.101243,2.276172,1.229883,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,2.0,2.042334
7850,9179.61.4593,-1.852610,5.425081,5.757202,-6.920499,2.561271,-1.741298,-1.045670,-1.061599,2.489897,...,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.10473,-0.723488,1.0,0.891251


In [ ]:
data_joined_test["Property Reference Id"] = data_joined_test["Property Reference Id"].astype(object)
data_joined_test.dtypes


<ipython-input-42-342d6ca3f86a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_joined_test["Property Reference Id"] = data_joined_test["Property Reference Id"].astype(object)


Property Reference Id     object
0                        float64
1                        float64
2                        float64
3                        float64
                          ...   
Geo_VS                   float64
Geo_ZG                   float64
Geo_ZH                   float64
Demand                   float64
Prediction               float64
Length: 70, dtype: object

In [ ]:
temp_dataset = data_joined_test.groupby(['Property Reference Id'])
tab = temp_dataset.mean()
display(tab.sample(5))

<ipython-input-43-36475d7a97ee>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tab = temp_dataset.mean()


,0,1,2,3,4,5,6,7,8,9,...,Geo_SZ,Geo_TG,Geo_TI,Geo_UR,Geo_VD,Geo_VS,Geo_ZG,Geo_ZH,Demand,Prediction
Property Reference Id,,,,,,,,,,,,,,,,,,,,,
7276.22.0065,-0.151769,4.691804,4.763517,-5.378359,2.089910,-1.122222,0.730841,1.588573,0.597662,5.687003,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,0.000000,0.891251
7977.22.5664,0.213037,4.904241,4.894507,-5.553480,2.216729,-0.663933,0.927539,1.938606,0.325356,5.876572,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,2.000000,1.670750
5888.22.9935,0.472074,5.079700,4.953190,-5.986607,2.689590,-1.171054,1.182277,2.387276,0.408414,5.914227,...,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.10473,-0.723488,2.000000,0.891251
6791.61.4266,0.016830,4.677358,4.784645,-5.300134,1.915972,-0.340583,0.822994,2.344957,0.440216,5.762862,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,4.333333,1.232676
9564.26.4593,-0.107945,5.242588,5.112111,-6.010555,2.570060,-0.540652,0.367418,1.305161,0.755894,6.135576,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,8.000000,2.948352


In [ ]:
data_joined_test.loc[data_joined_test.Prediction<=0,'Prediction']=10**(-0.05)
print("Final mean Poisson deviance(per ID):", mean_poisson_deviance(data_joined_test["Demand"], data_joined_test["Prediction"]))


Final mean Poisson deviance(per ID): 5.256561406907189
